
# Modelling & solving the multiple couriers planning problem

Matthieu Feraud and Marius Lesaulnier | Erasmus 2022 - 2023


---

## Installations and Imports

In [478]:
'''
!pip3 install z3-solver
!pip3 install utils
!pip3 install gurobipy
'''

'\n!pip3 install z3-solver\n!pip3 install utils\n!pip3 install gurobipy\n'

In [100]:
import json
import time
from math import log2, floor
from os import scandir, getcwd

import numpy as np
from gurobipy import Model, GRB, quicksum
from z3.z3 import *


In [176]:
filespath = getcwd() + '\\Instances\\instancesDat\\'
data_list = []

with scandir(filespath) as file_list:
    for filename in file_list:
        with open(filespath + filename.name) as f:
            lines = f.readlines()

            num_couriers = int(lines[0].strip())
            num_items = int(lines[1].strip())
            courier_loads = [int(i) for i in lines[2].strip().split()]
            item_sizes = [int(i) for i in lines[3].strip().split()]
            distance_matrix = [[int(j) for j in i.strip().split()] for i in lines[4:]]

            data_list.append({
                "m": num_couriers,
                "n": num_items,
                "l": courier_loads,
                "s": item_sizes,
                "D": distance_matrix
            })

In [9]:
data_list = sorted(data_list, key=lambda x: x['m'] + x['n'])
for i in range(20):
    data = data_list[i]
    m = data['m']  # num_couriers
    n = data['n']  # num_items
    print("id: " + str(i) + " m: " + str(m), "n: " + str(n), data['D'][0])

id: 0 m: 2 n: 3 [0, 21, 86, 99]
id: 1 m: 2 n: 6 [0, 3, 4, 5, 6, 6, 2]
id: 2 m: 3 n: 7 [0, 3, 3, 6, 5, 6, 6, 2]
id: 3 m: 6 n: 8 [0, 80, 131, 22, 41, 127, 87, 48, 113]
id: 4 m: 6 n: 9 [0, 199, 119, 28, 179, 77, 145, 61, 123, 87]
id: 5 m: 8 n: 10 [0, 56, 86, 77, 81, 128, 107, 154, 70, 93, 53]
id: 6 m: 8 n: 10 [0, 56, 86, 87, 81, 128, 107, 163, 166, 98, 93]
id: 7 m: 6 n: 17 [0, 20, 19, 28, 58, 48, 45, 32, 90, 61, 71, 59, 65, 46, 72, 51, 46, 66]
id: 8 m: 10 n: 13 [0, 49, 80, 59, 112, 79, 112, 187, 28, 47, 36, 69, 138, 54]
id: 9 m: 10 n: 13 [0, 21, 86, 14, 84, 72, 24, 54, 83, 70, 8, 91, 42, 57]
id: 10 m: 3 n: 47 [0, 60, 141, 22, 41, 137, 77, 48, 92, 105, 113, 103, 82, 15, 79, 24, 98, 69, 82, 30, 105, 89, 57, 94, 75, 50, 127, 16, 36, 77, 57, 70, 51, 101, 88, 38, 83, 108, 81, 124, 54, 131, 99, 70, 112, 162, 94, 64]
id: 11 m: 20 n: 47 [0, 59, 135, 21, 41, 132, 75, 48, 88, 102, 109, 101, 81, 15, 78, 24, 96, 67, 80, 30, 103, 87, 56, 93, 73, 49, 125, 15, 36, 76, 56, 68, 50, 98, 85, 37, 81, 106, 79

In [198]:
for i in range(20):
    data = data_list[i]
    m = data['m']  # num_couriers
    n = data['n']  # num_items
    print("id: " + str(i) + " m: " + str(m), "n: " + str(n), data['D'][0])

id: 0 m: 2 n: 6 [0, 3, 4, 5, 6, 6, 2]
id: 1 m: 6 n: 9 [0, 199, 119, 28, 179, 77, 145, 61, 123, 87]
id: 2 m: 3 n: 7 [0, 3, 3, 6, 5, 6, 6, 2]
id: 3 m: 8 n: 10 [0, 56, 86, 77, 81, 128, 107, 154, 70, 93, 53]
id: 4 m: 2 n: 3 [0, 21, 86, 99]
id: 5 m: 6 n: 8 [0, 80, 131, 22, 41, 127, 87, 48, 113]
id: 6 m: 6 n: 17 [0, 20, 19, 28, 58, 48, 45, 32, 90, 61, 71, 59, 65, 46, 72, 51, 46, 66]
id: 7 m: 8 n: 10 [0, 56, 86, 87, 81, 128, 107, 163, 166, 98, 93]
id: 8 m: 10 n: 13 [0, 49, 80, 59, 112, 79, 112, 187, 28, 47, 36, 69, 138, 54]
id: 9 m: 10 n: 13 [0, 21, 86, 14, 84, 72, 24, 54, 83, 70, 8, 91, 42, 57]
id: 10 m: 20 n: 143 [0, 47, 68, 167, 65, 46, 131, 123, 118, 129, 85, 113, 83, 16, 52, 124, 56, 183, 82, 81, 13, 128, 117, 45, 142, 128, 119, 125, 132, 47, 65, 20, 30, 124, 135, 19, 60, 124, 103, 115, 20, 137, 41, 73, 42, 41, 94, 43, 112, 125, 181, 25, 111, 65, 50, 37, 143, 32, 65, 38, 133, 25, 137, 6, 72, 92, 51, 49, 91, 47, 59, 55, 119, 105, 69, 5, 61, 86, 35, 124, 46, 143, 8, 117, 1, 41, 51, 80, 53,

In [178]:
data = data_list[1]
m = data['m']  # num_couriers
n = data['n']  # num_items
l = data['l']  # courier_loads
s = data['s']  # item_sizes
D = data['D']  # distance_matrix

## SAT MODEL

### Defining the decisions variables and constraints

In [208]:
def define_decision_variables(num_couriers, num_items):
    #These variables control the couriers’ movements and ensure that they follow the optimal path
    x = [[Bool(f"x_{i}_{j}") for j in range(num_items + 1)] for i in range(num_items + 1)]
    #These variables signal ’true’ when courier k is allocated to node i
    v = [[Bool(f"v_{i}_{k}") for k in range(num_couriers)] for i in range(num_items)]

    num_bits = int(log2(num_items)) + 1
    #These variables purpose is to encode the sequence of node visits in binary notation
    u = [[Bool(f"u_{i}_{k}") for k in range(num_bits)] for i in range(num_items)]
    return x, v, u

In [209]:
def exactly_one(boolean_vars):
    return And(AtMost(*boolean_vars, 1), AtLeast(*boolean_vars, 1))


def exactly(boolean_vars, nb):
    return And(AtMost(*boolean_vars, nb), AtLeast(*boolean_vars, nb))


def binary_increment(a, b):
    constraints = []
    carry = {}
    num_digits = len(a)

    constraints.append(b[0] == Not(a[0]))
    constraints.append(b[1] == Or(And(a[1], Not(a[0])), And(Not(a[1]), a[0])))
    carry[1] = a[0]

    for i in range(2, num_digits):
        carry[i] = And(a[i - 1], carry[i - 1])
        constraints.append(b[i] == Or(And(a[i], Not(carry[i])), And(Not(a[i]), carry[i])))

    return And(constraints)


In [218]:
def add_constraints(solver, x, v, u, num_couriers, num_items, max_loads, item_sizes, distance_matrix, max_distance):
    # No route from itself to itself
    solver.add([x[i][i] == False for i in range(num_items + 1)])

    # Each location must be visited exactly once
    solver.add([exactly_one([x[i][j] for j in range(num_items + 1)]) for i in range(num_items)])
    solver.add([exactly_one([x[i][j] for i in range(num_items + 1)]) for j in range(num_items)])

    # At most num_couriers couriers can start from the origin and return to the origin
    solver.add(Sum([x[num_items][j] for j in range(num_items + 1)]) == num_couriers)
    solver.add(Sum([x[i][num_items] for i in range(num_items + 1)]) == num_couriers)

    #solver.add(AtMost(*[x[num_items + 1][j] for j in range(num_items)], num_couriers))
    #solver.add(AtMost(*[x[i][num_items + 1] for i in range(num_items)], num_couriers))

    # Each courier can start from the origin at most once. This means a courier can't start delivering from the origin more than once.
    #solver.add([AtMost(*[And(v[j][k], x[num_items][j]) for j in range(num_items)], 1) for k in range(num_couriers)])
    # If courier i goes from location j to location k, then courier i must also carry item j
    solver.add([Implies(x[i][j], v[i][k] == v[j][k]) for k in range(num_couriers) for i in range(num_items) for j in
                range(num_items)])

    # Each item must be carried by exactly one courier
    solver.add([exactly_one(v[i]) for i in range(num_items)])

    # solver.add(
    #    [Implies(And(x[num_items][i], x[i][j]), i < j) for i in range(num_items) for j in range(num_items) if i != j])

    for i in range(len(distance_matrix)-1):
        for j in range(len(distance_matrix)-1):
            if distance_matrix[i][j] == distance_matrix[j][i] and i < j:
                solver.add(x[j][i] == False)

    # The order of visiting locations must be consistent with the binary representations
    for j in range(num_items):
        for i in range(num_items):
            if i != j:
                solver.add(Implies(x[i][j], binary_increment(u[i], u[j])))

    # The total distance travelled by the couriers must not exceed the max distance
    solver.add(
        PbLe([(x[i][j], distance_matrix[i, j]) for j in range(num_items + 1) for i in range(num_items + 1) if i != j],
             max_distance - 1))

    # The total size of items carried by each courier must not exceed the courier's max load
    for k in range(num_couriers):
        assert len(v) == num_items, "The list v doesn't have num_items elements."
        for sublist in v:
            assert len(sublist) == num_couriers, "One of the sublists in v doesn't have num_couriers elements."
        assert len(item_sizes) == num_items, "The list item_sizes doesn't have num_items elements."
        assert len(max_loads) == num_couriers, "The list max_loads doesn't have num_couriers elements."
        solver.add(PbLe([(v[i][k], item_sizes[i]) for i in range(num_items)], max_loads[k]))


#for i in range(num_items):
#   solver.add([Implies(v[i][k2], v[i][k1]) for k1 in range(num_couriers) for k2 in range(num_couriers) if k1 > k2])

# Symmetry-breaking constraint: enforce a specific ordering of courier assignments
#for i in range(num_items):
#    solver.add([Implies(v[i][k2], v[i][k1]) for k1 in range(num_couriers) for k2 in range(num_couriers) if k1 > k2])

# Symmetry-breaking constraint: If courier i goes from the origin to location j, then i must be less than j, order of node assignment
#solver.add(
#    [Implies(And(x[num_items][i], x[i][j]), i < j) for i in range(num_items) for j in range(num_items) if i != j])


### Formulating the Optimization Problem

In [217]:
def define_problem(num_couriers, num_items, max_loads, item_sizes, distance_matrix, max_distance=None):
    distance_matrix = np.array(distance_matrix)

    if max_distance is None:
        max_distance = int(np.sum(distance_matrix))

    #solver = Optimize()
    solver = SolverFor("QF_BV")

    x, v, u = define_decision_variables(num_couriers, num_items)

    add_constraints(solver, x, v, u, num_couriers, num_items, max_loads, item_sizes, distance_matrix, max_distance)

    #total_distance = Sum([distance_matrix[i, j] * x[i][j] for j in range(num_items + 1) for i in range(num_items + 1) if i != j])
    #solver.minimize(total_distance)
    return solver, x, v




### Interpreting and Displaying Optimization Results

In [219]:
def handle_solution(solver, x, v, num_couriers, num_items, distance_matrix):
    solver.set("timeout", 60)  # timeout is in milliseconds
    model = solver.model()
    route_matrix = [[model.evaluate(x[i][j]) for j in range(num_items + 1)] for i in range(num_items + 1)]
    item_courier_matrix = [[model.evaluate(v[i][k]) for k in range(num_couriers)] for i in range(num_items)]
    num_nodes = len(route_matrix) - 1
    routes = {}
    courier_distance = {}

    for origin_node in range(num_nodes + 1):
        if route_matrix[num_nodes][origin_node]:  # if there is a route from the last node to the current node
            courier_path = [num_nodes, origin_node]  # start a new path
            courier_id = item_courier_matrix[origin_node].index(True)  # assign a courier to this path
            current_node = origin_node  # initialize the current node
            courier_distance[courier_id] = distance_matrix[num_nodes][
                origin_node]  # initialize the courier's total distance

            # Continue the path until we reach back to the last node
            while current_node != num_nodes:
                if courier_id != item_courier_matrix[current_node].index(True):
                    print('Some error occurred with courier', courier_id)
                for next_node in range(num_nodes + 1):
                    if current_node != next_node and route_matrix[current_node][next_node]:
                        courier_path.append(next_node)  # append the next node to the path
                        courier_distance[courier_id] += distance_matrix[current_node][
                            next_node]  # update the courier's total distance
                        current_node = next_node  # move to the next node
                        break
            routes[courier_id] = courier_path  # store the courier's path
    total_distance = 0
    paths = []

    for courier_id, route in routes.items():
        path_display = " > ".join([str((node + 1) % (num_nodes + 1)) for node in route])
        distance_display = f"{courier_distance[courier_id]:,}"
        paths.append((courier_id + 1, path_display, distance_display))
        total_distance += courier_distance[courier_id]

    # Print the courier paths
    print("Courier Paths:")
    print("-" * 60)
    print("{:<10s} {:<30s} {:<15s}".format("Courier", "Path", "Distance"))
    print("-" * 60)
    for courier_id, path_display, distance_display in paths:
        print("{:<10d} {:<30s} {:<15s}".format(courier_id, path_display, distance_display))
    print("-" * 60)

    # Print the total distance
    print("Total distance:", f"{total_distance:,}")
    return total_distance


### Executing Item Assignment and Route Optimization for the SAT Model

In [220]:
def assign(num_couriers, num_items, max_loads, item_sizes, distance_matrix):
    if sum(item_sizes) > sum(max_loads):
        print("Total weight of all items surpasses possible weight transportable")
        return 0

    remaining_time = 300
    solver, x, v = define_problem(num_couriers, num_items, max_loads, item_sizes, distance_matrix)

    if solver.check() == sat:
        total_distance = handle_solution(solver, x, v, num_couriers, num_items, distance_matrix)
        print(f"Initial solution found with total distance: {total_distance}")

        while remaining_time > 0:
            print(f"Looking for solutions with total distance < {total_distance}")
            time_before = time.time()
            solver, x, v = define_problem(num_couriers, num_items, max_loads, item_sizes, distance_matrix,
                                          total_distance)

            if solver.check() == sat:
                total_distance = handle_solution(solver, x, v, num_couriers, num_items, distance_matrix)
                elapsed_time = time.time() - time_before
                remaining_time -= elapsed_time
                print(f"Elapsed time: {elapsed_time:.2f} seconds")
                print(f"Remaining time: {remaining_time:.2f} seconds")
            else:
                print("No more solutions found.")
                break

        if remaining_time <= 0:
            print("Time limit exceeded")
        return 0
    else:
        print("Failed to find a solution")
        return 0


In [221]:
assign(m, n, l, s, D)

IndexError: index 10 is out of bounds for axis 0 with size 10

## One symmetry breaking constraints : Courier assignment



In [490]:
def add_constraints(solver, x, v, u, num_couriers, num_items, max_loads, item_sizes, distance_matrix, max_distance):
    # No route from itself to itself
    solver.add([x[i][i] == False for i in range(num_items + 1)])

    # Each location must be visited exactly once
    solver.add([exactly_one([x[i][j] for j in range(num_items + 1)]) for i in range(num_items)])
    solver.add([exactly_one([x[i][j] for i in range(num_items + 1)]) for j in range(num_items)])

    # At most num_couriers couriers can start from the origin and return to the origin
    solver.add(AtMost(*[x[num_items][j] for j in range(num_items)], num_couriers))
    solver.add(AtMost(*[x[i][num_items] for i in range(num_items)], num_couriers))

    # Each courier can start from the origin at most once. This means a courier can't start delivering from the origin more than once.
    solver.add([AtMost(*[And(v[j][k], x[num_items][j]) for j in range(num_items)], 1) for k in range(num_couriers)])
    # If courier i goes from location j to location k, then courier i must also carry item j
    solver.add([Implies(x[i][j], v[i][k] == v[j][k]) for k in range(num_couriers) for i in range(num_items) for j in
                range(num_items)])

    # Each item must be carried by exactly one courier
    solver.add([exactly_one(v[i]) for i in range(num_items)])

    # The total size of items carried by each courier must not exceed the courier's max load
    for k in range(num_couriers):
        assert len(v) == num_items, "The list v doesn't have num_items elements."
        for sublist in v:
            assert len(sublist) == num_couriers, "One of the sublists in v doesn't have num_couriers elements."
        assert len(item_sizes) == num_items, "The list item_sizes doesn't have num_items elements."
        assert len(max_loads) == num_couriers, "The list max_loads doesn't have num_couriers elements."
        solver.add(PbLe([(v[i][k], item_sizes[i]) for i in range(num_items)], max_loads[k]))

    # The total distance travelled by the couriers must not exceed the max distance
    solver.add(
        PbLe([(x[i][j], distance_matrix[i, j]) for j in range(num_items + 1) for i in range(num_items + 1) if i != j],
             max_distance - 1))

    # The order of visiting locations must be consistent with the binary representations
    for j in range(num_items):
        for i in range(num_items):
            if i != j:
                solver.add(Implies(x[i][j], binary_increment(u[i], u[j])))

    # Symmetry-breaking constraint: enforce a specific ordering of courier assignments
    for i in range(num_items):
        solver.add([Implies(v[i][k2], v[i][k1]) for k1 in range(num_couriers) for k2 in range(num_couriers) if k1 > k2])
    '''
    # Symmetry-breaking constraint: If courier i goes from the origin to location j, then i must be less than j, order of node assignment
    solver.add([Implies(And(x[num_items][i],x[i][j]),i<j) for i in range(num_items) for j in range(num_items) if i!=j])
    '''


assign(m, n, l, s, D)

Failed to find a solution


0

## One symmetry breaking constraints : Node assignment


In [491]:
def add_constraints(solver, x, v, u, num_couriers, num_items, max_loads, item_sizes, distance_matrix, max_distance):
    # No route from itself to itself
    solver.add([x[i][i] == False for i in range(num_items + 1)])

    # Each location must be visited exactly once
    solver.add([exactly_one([x[i][j] for j in range(num_items + 1)]) for i in range(num_items)])
    solver.add([exactly_one([x[i][j] for i in range(num_items + 1)]) for j in range(num_items)])

    # At most num_couriers couriers can start from the origin and return to the origin
    solver.add(AtMost(*[x[num_items][j] for j in range(num_items)], num_couriers))
    solver.add(AtMost(*[x[i][num_items] for i in range(num_items)], num_couriers))

    # Each courier can start from the origin at most once. This means a courier can't start delivering from the origin more than once.
    solver.add([AtMost(*[And(v[j][k], x[num_items][j]) for j in range(num_items)], 1) for k in range(num_couriers)])
    # If courier i goes from location j to location k, then courier i must also carry item j
    solver.add([Implies(x[i][j], v[i][k] == v[j][k]) for k in range(num_couriers) for i in range(num_items) for j in
                range(num_items)])

    # Each item must be carried by exactly one courier
    solver.add([exactly_one(v[i]) for i in range(num_items)])

    # The total size of items carried by each courier must not exceed the courier's max load
    for k in range(num_couriers):
        assert len(v) == num_items, "The list v doesn't have num_items elements."
        for sublist in v:
            assert len(sublist) == num_couriers, "One of the sublists in v doesn't have num_couriers elements."
        assert len(item_sizes) == num_items, "The list item_sizes doesn't have num_items elements."
        assert len(max_loads) == num_couriers, "The list max_loads doesn't have num_couriers elements."
        solver.add(PbLe([(v[i][k], item_sizes[i]) for i in range(num_items)], max_loads[k]))

    # The total distance travelled by the couriers must not exceed the max distance
    solver.add(
        PbLe([(x[i][j], distance_matrix[i, j]) for j in range(num_items + 1) for i in range(num_items + 1) if i != j],
             max_distance - 1))

    # The order of visiting locations must be consistent with the binary representations
    for j in range(num_items):
        for i in range(num_items):
            if i != j:
                solver.add(Implies(x[i][j], binary_increment(u[i], u[j])))

    '''
    # Symmetry-breaking constraint: enforce a specific ordering of courier assignments
    for i in range(num_items):
        solver.add([Implies(v[i][k2], v[i][k1]) for k1 in range(num_couriers) for k2 in range(num_couriers) if k1 > k2])
    '''
    # Symmetry-breaking constraint: If courier i goes from the origin to location j, then i must be less than j, order of node assignment
    solver.add(
        [Implies(And(x[num_items][i], x[i][j]), i < j) for i in range(num_items) for j in range(num_items) if i != j])


assign(m, n, l, s, D)

Courier Paths:
------------------------------------------------------------
Courier    Path                           Distance       
------------------------------------------------------------
20         0 > 3 > 28 > 1 > 32 > 0        599            
16         0 > 6 > 58 > 51 > 0            248            
6          0 > 8 > 0                      94             
12         0 > 11 > 55 > 54 > 70 > 9 > 30 > 0 284            
19         0 > 15 > 0                     136            
4          0 > 18 > 69 > 14 > 0           206            
5          0 > 24 > 25 > 22 > 36 > 21 > 4 > 31 > 0 603            
7          0 > 27 > 33 > 10 > 7 > 71 > 57 > 20 > 19 > 0 877            
10         0 > 29 > 44 > 0                146            
1          0 > 34 > 0                     104            
18         0 > 35 > 0                     110            
2          0 > 38 > 63 > 26 > 13 > 46 > 5 > 2 > 0 695            
14         0 > 39 > 42 > 40 > 0           234            
3          0 > 4

0

## No symmetry breaking constraints

In [492]:
def add_constraints(solver, x, v, u, num_couriers, num_items, max_loads, item_sizes, distance_matrix, max_distance):
    # No route from itself to itself
    solver.add([x[i][i] == False for i in range(num_items + 1)])

    # Each location must be visited exactly once
    solver.add([exactly_one([x[i][j] for j in range(num_items + 1)]) for i in range(num_items)])
    solver.add([exactly_one([x[i][j] for i in range(num_items + 1)]) for j in range(num_items)])

    # At most num_couriers couriers can start from the origin and return to the origin
    solver.add(AtMost(*[x[num_items][j] for j in range(num_items)], num_couriers))
    solver.add(AtMost(*[x[i][num_items] for i in range(num_items)], num_couriers))

    # Each courier can start from the origin at most once. This means a courier can't start delivering from the origin more than once.
    solver.add([AtMost(*[And(v[j][k], x[num_items][j]) for j in range(num_items)], 1) for k in range(num_couriers)])
    # If courier i goes from location j to location k, then courier i must also carry item j
    solver.add([Implies(x[i][j], v[i][k] == v[j][k]) for k in range(num_couriers) for i in range(num_items) for j in
                range(num_items)])

    # Each item must be carried by exactly one courier
    solver.add([exactly_one(v[i]) for i in range(num_items)])

    # The total size of items carried by each courier must not exceed the courier's max load
    for k in range(num_couriers):
        assert len(v) == num_items, "The list v doesn't have num_items elements."
        for sublist in v:
            assert len(sublist) == num_couriers, "One of the sublists in v doesn't have num_couriers elements."
        assert len(item_sizes) == num_items, "The list item_sizes doesn't have num_items elements."
        assert len(max_loads) == num_couriers, "The list max_loads doesn't have num_couriers elements."
        solver.add(PbLe([(v[i][k], item_sizes[i]) for i in range(num_items)], max_loads[k]))

    # The total distance travelled by the couriers must not exceed the max distance
    solver.add(
        PbLe([(x[i][j], distance_matrix[i, j]) for j in range(num_items + 1) for i in range(num_items + 1) if i != j],
             max_distance - 1))

    # The order of visiting locations must be consistent with the binary representations
    for j in range(num_items):
        for i in range(num_items):
            if i != j:
                solver.add(Implies(x[i][j], binary_increment(u[i], u[j])))


assign(m, n, l, s, D)

Courier Paths:
------------------------------------------------------------
Courier    Path                           Distance       
------------------------------------------------------------
5          0 > 16 > 71 > 64 > 69 > 41 > 51 > 9 > 0 735            
4          0 > 20 > 68 > 57 > 38 > 0      580            
2          0 > 27 > 15 > 13 > 1 > 2 > 39 > 61 > 53 > 29 > 47 > 19 > 45 > 55 > 6 > 44 > 26 > 14 > 66 > 0 1,705          
12         0 > 31 > 10 > 35 > 34 > 0      272            
19         0 > 32 > 0                     112            
6          0 > 33 > 3 > 43 > 25 > 28 > 23 > 7 > 60 > 0 875            
1          0 > 36 > 56 > 54 > 30 > 0      235            
18         0 > 48 > 50 > 24 > 49 > 0      355            
3          0 > 58 > 21 > 18 > 4 > 67 > 22 > 0 422            
8          0 > 59 > 63 > 5 > 52 > 8 > 11 > 40 > 46 > 17 > 12 > 65 > 37 > 62 > 42 > 70 > 0 992            
------------------------------------------------------------
Total distance: 6,283
Initia

0

## MIP Model

### Defining the constraints

In [19]:
# Function to add constraints to the model
def add_constraints(mdl, num_items, num_couriers, Vertices, Items, Arcs, x, l, courier_assignment, aux_vars, item_loads,
                    max_loads):
    # Only one connection into point i
    mdl.addConstrs(quicksum(x[i, j] for j in Vertices if j != i) == 1 for i in Items)

    # Only one edge out of point i
    mdl.addConstrs(quicksum(x[i, j] for i in Vertices if i != j) == 1 for j in Items)

    # No more than "num_couriers" departures from the depot
    mdl.addConstr(quicksum(x[0, j] for j in Items) == num_couriers)

    # Same number of couriers leaving and arriving to the depot
    mdl.addConstr(quicksum(x[0, j] for j in Items) == quicksum(x[j, 0] for j in Items))

    # Sub-tour elimination
    mdl.addConstr(l[0] == 1)
    mdl.addConstrs((l[i] + x[i, j]) <= (l[j] + num_items * (1 - x[i, j])) for i, j in Arcs if j != 0)

    # Constraints related to courier
    for k in range(num_couriers):
        mdl.addConstrs((aux_vars[i][k] <= courier_assignment[i][k]) for i in Items)
        mdl.addConstrs((aux_vars[i][k] <= x[0, i]) for i in Items)
        mdl.addConstrs((aux_vars[i][k] >= courier_assignment[i][k] + x[0, i] - 1) for i in Items)
        mdl.addConstr(quicksum(aux_vars[i][k] for i in Items) <= 1)

        # If x[i,j] = True, i,j share the same courier
        for i, j in Arcs:
            if i != 0 and j != 0:
                mdl.addConstr(x[i, j] + courier_assignment[i][k] - courier_assignment[j][k] <= 1)
                mdl.addConstr(x[i, j] - courier_assignment[i][k] + courier_assignment[j][k] <= 1)

    # Every item should be visited by exactly one courier
    mdl.addConstrs(quicksum(courier_assignment[i][k] for k in range(num_couriers)) == 1 for i in Items)

    # The total load of items served by each courier cannot exceed the courier's capacity
    mdl.addConstrs(quicksum(item_loads[i - 1] * courier_assignment[i][k] for i in Items) <= max_loads[k] for k in
                   range(num_couriers))

### Creating the model

In [30]:
# Function to create the Gurobi model
def create_model(num_items, num_couriers, item_loads, item_sizes, distance_matrix):
    Items = [i for i in range(1, num_items + 1)]
    # Define the set of vertices, including the depot
    Vertices = [0] + Items
    Arcs = [(i, j) for i in Vertices for j in Vertices if i != j]

    Couriers_encoding = [i for i in range(num_couriers)]
    courier_assignment = {}
    aux_vars = {}

    mdl = Model('CVRP')

    # Define variables
    x = mdl.addVars(Arcs, vtype=GRB.BINARY)
    l = mdl.addVars(Vertices, lb=0, ub=num_items, vtype=GRB.INTEGER)
    for i in range(1, num_items + 1):
        courier_assignment[i] = mdl.addVars(Couriers_encoding, vtype=GRB.BINARY)
        aux_vars[i] = mdl.addVars(Couriers_encoding, vtype=GRB.INTEGER)

    # Objective function
    mdl.modelSense = GRB.MINIMIZE
    mdl.setObjective(quicksum(x[i, j] * distance_matrix[i, j] for (i, j) in Arcs))

    # Constraints
    add_constraints(mdl, num_items, num_couriers, Vertices, Items, Arcs, x, l, courier_assignment, aux_vars, item_loads,
                    item_sizes)

    # Solver settings
    mdl.params.ScaleFlag = 1
    mdl.params.MIPFocus = 1
    mdl.Params.TimeLimit = 300

    return mdl, x, courier_assignment, Items, Vertices



### Displaying the Optimization Results

In [71]:
def handle_solution(mdl, x, courier_assignment, Items, Vertices, num_couriers):
    tours = {}
    for i in Items:
        try:
            x[(0, i)].x > 0.1
        except AttributeError:
            print("Total distance: N/A")
            break
        if x[(0, i)].x > 0.9:
            for k in range(num_couriers):
                if int(str(courier_assignment[i][k])[-6:-4]) > 0:
                    temp = k
            temp += 1
            tours[temp] = []
            path = [0, i]
            while i != 0:
                j = i
                for k in Vertices:
                    if j != k and x[(j, k)].x > 0.9:
                        path.append(k)
                        i = k
            tours[temp].append(path)

    sols = []
    for key in tours.keys():
        sol = []
        path_str = str(tours[key])
        path_str = path_str.replace("[", "")
        path_str = path_str.replace("]", "")
        path_str = path_str.replace(", ", " > ")
        tours[key] = path_str

        for i in tours[key]:
            if i != '0' and i != '>' and i != ' ':
                sol.append(int(i))
        sols.append(sol)

    tour_ids = [i for i in tours.keys()]
    print("Planned Tours:")

    for tour_id in sorted(tour_ids):
        print("Tour " + str(tour_id) + ":")
        print(tours[tour_id])

    print("---------------------------")
    try:
        print("Total distance:", int(mdl.ObjVal))
    except OverflowError:
        print("Total distance: N/A")

    return int(mdl.ObjVal), sols

### Executing Item Assignment and Route Optimization for the MIP Model

In [55]:
def assign(num_items, num_couriers, item_loads, max_loads, distance_matrix):
    # Create distance matrix
    Dist = np.array(distance_matrix)

    # Create Gurobi model
    mdl, x, courier_assignment, Items, Vertices = create_model(num_items, num_couriers, item_loads, max_loads, Dist)

    # Optimize model
    mdl.optimize()

    # Print solution
    obj, sols = handle_solution(mdl, x, courier_assignment, Items, Vertices, num_couriers)

    return obj, sols


assign(n, m, s, l, D)

Set parameter ScaleFlag to value 1
Set parameter MIPFocus to value 1
Set parameter TimeLimit to value 300
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 5700U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads



GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license

In [80]:
path_res = getcwd() + "\\res\\MIP\\"

for i in range(11, 21):
    print("-" * 5 + str(i) + "-" * 5)
    data = data_list[i]
    m = data['m']  # num_couriers
    n = data['n']  # num_items
    l = data['l']  # courier_loads
    s = data['s']  # item_sizes
    D = data['D']  # distance_matrix

    out = {}
    data_name = "0" + str(i + 1) if i < 10 else str(i + 1)

    time_before = time.time()
    obj, tours = assign(n, m, s, l, D)
    out["time"] = floor((time.time() - time_before) / 1000)
    out["obj"] = obj
    out["sol"] = tours

    with open(path_res + data_name + ".txt", 'w') as f:
        json.dump(out, f, ensure_ascii=False, indent=4)
    #print(tours)

-----11-----
Set parameter ScaleFlag to value 1
Set parameter MIPFocus to value 1
Set parameter TimeLimit to value 300
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 5700U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads



GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license